In [ ]:
import pandas as pd
import numpy as np

# Cargar el conjunto de datos de entrenamiento 
df_house = pd.read_csv('train.csv')

# Inspección inicial: ver las primeras filas para entender las columnas
print("--- Primeras 5 Filas del Dataset ---")
print(df_house.head())

# Inspección de información: ver tipos de datos y el gran desafío de nulos
print("\n--- Información General y Conteo de Nulos ---")
df_house.info()

# Contar el total de valores nulos por columna
nulos_totales = df_house.isnull().sum()

# Filtrar solo las columnas que tienen al menos un valor nulo y calcular el porcentaje
nulos_conteo = nulos_totales[nulos_totales > 0].sort_values(ascending=False)
nulos_porcentaje = (nulos_conteo / len(df_house)) * 100

# Crear un DataFrame para mostrar el resultado
df_nulos = pd.DataFrame({
    'Total Nulos': nulos_conteo,
    'Porcentaje (%)': nulos_porcentaje.round(2)
})

# Imprimir el resultado para ver el ranking
print("\n--- Columnas con Valores Nulos (NaN) y su Porcentaje ---")
print(df_nulos)

# Lista de columnas categóricas donde NaN significa 'None' (Ausencia de la característica)
cols_none_imputation = [
    'PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu',
    'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
    'BsmtExposure', 'BsmtFinType2', 'BsmtFinType1', 'BsmtCond', 'BsmtQual',
    'MasVnrType', 'MSZoning', 'Utilities', 'Exterior1st', 'Exterior2nd',
    'KitchenQual', 'Functional', 'SaleType', 'Electrical', 'Foundation',
    'Heating', 'HeatingQC', 'CentralAir', 'PavedDrive'
]

# Rellenamos los NaNs con la cadena 'None'
for col in cols_none_imputation:
    if col in df_house.columns:
        df_house[col] = df_house[col].fillna('None')

# Verificamos si logramos eliminar los nulos más grandes
print("\n--- Verificación: Top 5 de Nulos después de la imputación 'None' ---")
print(df_house.isnull().sum().sort_values(ascending=False).head())

# Lista de columnas numéricas que aún tienen NaNs
cols_median_imputation = ['LotFrontage', 'GarageYrBlt', 'MasVnrArea']

# Rellenamos los NaNs con la mediana de cada columna
for col in cols_median_imputation:
    mediana = df_house[col].median()
    df_house[col] = df_house[col].fillna(mediana)

# Finalmente, rellenamos el único valor nulo que queda en 'Electrical' con la moda
df_house['Electrical'] = df_house['Electrical'].fillna(df_house['Electrical'].mode()[0])

# Verificación final: contamos cuántos nulos quedan en todo el DataFrame
total_nulos_final = df_house.isnull().sum().sum()

print("\n--- ¡Verificación Final de Nulos! ---")
print(f"Total de valores nulos restantes en todo el DataFrame: {total_nulos_final}")

import numpy as np

# 1. Aplicar la Transformación Logarítmica a la Variable Objetivo
# Esto reduce el sesgo y mejora la linealidad para la regresión.
# Se usa np.log1p (log(1+x)) para manejar el valor 0 de forma segura.
df_house['SalePrice_Log'] = np.log1p(df_house['SalePrice'])

# 2. Eliminar la variable original 'SalePrice' y el identificador 'Id'
df_model = df_house.drop(['SalePrice', 'Id'], axis=1)

# 3. Conversión de Variables Categóricas (Dummies)
# Convertimos todas las columnas restantes de tipo 'object' (texto) a números.
df_model = pd.get_dummies(df_model)

# 4. Verificación de la Matriz Final
print("\n--- Verificación Final de la Matriz para el Modelo ---")
print(f"Número de filas: {df_model.shape[0]}")
print(f"Número de columnas (Features + Target): {df_model.shape[1]}")
print(f"Tipos de datos únicos restantes: {df_model.dtypes.apply(lambda x: x.name).unique()}")
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

# 1. Definición de X e Y
Y = df_model['SalePrice_Log'] # Variable Objetivo transformada
# X son todas las demás columnas, excepto la variable objetivo
X = df_model.drop('SalePrice_Log', axis=1)

# 2. División de Datos (80% entrenamiento, 20% prueba)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# 3. Entrenar el Modelo de Regresión Ridge
print("\nEntrenando Modelo de Regresión Ridge...")
# Alpha es el término de penalización (regularización). Usamos un valor inicial.
modelo_ridge = Ridge(alpha=10, random_state=42)
modelo_ridge.fit(X_train, Y_train)

# 4. Evaluación del Modelo (MSE)
Y_pred = modelo_ridge.predict(X_test)
# Usamos RMSE (Root Mean Squared Error) para evaluar la calidad de la predicción
# Lo elevamos al cuadrado para devolverlo a la escala original del logaritmo (exponente)
rmse = np.sqrt(mean_squared_error(Y_test, Y_pred))

print(f"\n✅ ¡RESULTADO FINAL! RMSE del modelo (en escala logarítmica): {rmse:.4f}")

# Importamos la librería que necesitamos
import numpy as np
import pandas as pd

# Creamos un DataFrame para los coeficientes
# El modelo Ridge tiene el atributo .coef_
df_coeficientes_house = pd.DataFrame({
    'Característica': X.columns,
    'Coeficiente': modelo_ridge.coef_
})

# Ordenamos por valor absoluto para encontrar las más influyentes
df_coeficientes_house['Impacto Absoluto'] = abs(df_coeficientes_house['Coeficiente'])
df_coeficientes_house = df_coeficientes_house.sort_values(by='Impacto Absoluto', ascending=False)

# Imprimimos el Top 10 de características más influyentes
print("\n--- Top 10 Características Más Influyentes en el Precio ---")
print(df_coeficientes_house[['Característica', 'Coeficiente']].head(10).round(4))

import matplotlib.pyplot as plt
import seaborn as sns

# Nota: Usamos el DataFrame 'df_coeficientes_house' creado en el paso anterior.
# Necesitamos solo las 10 filas principales (las más influyentes).
df_top_10 = df_coeficientes_house.head(10)

plt.figure(figsize=(10, 6))
# Usamos un gráfico de barras horizontal (barplot)
# La Característica va en el eje Y y el Coeficiente (el valor de impacto) en el eje X
sns.barplot(x='Coeficiente', y='Característica', data=df_top_10, palette='RdBu')

plt.title('Top 10 Coeficientes (Impacto) en el Precio de Venta (Regresión Ridge)')
plt.xlabel('Coeficiente (Valor Positivo = Sube el Precio / Negativo = Baja el Precio)')
plt.ylabel('Característica')
plt.grid(axis='x', linestyle='--')
plt.show()

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
import numpy as np

# Definir la cuadrícula de valores a probar (Grid) para el parámetro 'alpha'
# Estos valores representan diferentes fuerzas de regularización.
parametros = {'alpha': [0.1, 1.0, 5.0, 10.0, 50.0, 100.0, 200.0, 500.0]}

# 1. Configurar GridSearchCV
# Usamos el modelo Ridge y definimos que el Grid Search debe buscar el 'neg_mean_squared_error' (error cuadrático medio negativo).
# El 'neg' se usa porque Grid Search siempre busca maximizar (y queremos minimizar el error, por lo que maximizar su negativo).
# cv=5 indica 5-fold cross-validation.
grid_search = GridSearchCV(
    estimator=Ridge(random_state=42),
    param_grid=parametros,
    scoring='neg_mean_squared_error',
    cv=5,
    verbose=1,
    n_jobs=-1  # Usa todos los núcleos del procesador
)

# 2. Ejecutar el Grid Search (entrenamiento)
print("\nIniciando Grid Search para encontrar el alpha óptimo...")
grid_search.fit(X_train, Y_train)

# 3. Obtener el mejor resultado
mejor_alpha = grid_search.best_params_['alpha']
mejor_score = np.sqrt(-grid_search.best_score_) # Convertir el score negativo a RMSE positivo

print("\n--- Resultados de la Optimización ---")
print(f"Alpha óptimo encontrado por Grid Search: {mejor_alpha}")
print(f"Mejor RMSE de Validación Cruzada (con el alpha óptimo): {mejor_score:.4f}")